# Day 23
## Part 1

I need to get the hallway and 4 burrows with data.

In [81]:
import collections
import numpy as np
num_to_species = {1: "A", 2: "B", 3: "C", 4: "D"}

lines = []
with open("ex1.txt") as f:
    hallway_index = 1000
    burrows = collections.defaultdict(list)
    burrow_connection = {}
    for index, line in enumerate(f.readlines()):
        # create field
        line_list = [char for char in line.strip()]
        if lines:
            while len(line_list) < len(lines[0]):
                line_list = ["#"] + line_list + ["#"]
        lines.append(line_list)

        # create other stuff
        line = line.strip()
        if "." in line:
            hallway_index = index
            hallway = ["."] * line.count(".")
        if index > hallway_index:
            n_burrow = 1
            for char_index, char in enumerate(line):
                if char != "#":
                    if n_burrow not in burrow_connection:
                        burrow_connection[num_to_species[n_burrow]] = char_index - 1
                    burrows[num_to_species[n_burrow]].append(char)
                    n_burrow += 1
                
            
print(hallway)
for k, v in burrows.items():
    print(k, v)
for k, v in burrow_connection.items():
    print(k, v)
    
field = np.array(lines)
field = field[1:-1, 1:-1]
print(field)

['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
A ['B', 'A']
B ['C', 'D']
C ['B', 'C']
D ['D', 'A']
A 0
B 2
C 4
D 6
[['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['#' '#' 'B' '#' 'C' '#' 'B' '#' 'D' '#' '#']
 ['#' '#' 'A' '#' 'D' '#' 'C' '#' 'A' '#' '#']]


In [1]:
def check_won(field):
    if all((all(field[1:, 2] == "A"), all(field[1:, 4] == "B"), 
            all(field[1:, 6] == "C"), all(field[1:, 8] == "D"))):
        return True
    else:
        return False

In [2]:
def check_neighbours(space, field, neighbour_set):
    try:
        current_value = field[space[0], space[1]]
    except IndexError:
        # print("checking neighbours OOB", space)
        return neighbour_set
    
    # print("checking neighbours haha", space, current_value) 
    if space in neighbour_set or current_value != ".":
        return neighbour_set
    else:
        neighbour_set.add(space)
        if space[0] == 0:
            # in hallway
            if space[1] == 0:
                # only neighbour is to the right
                neighbour_set = check_neighbours((0, 1), field, neighbour_set)
            elif space[1] == field.shape[1] - 1:
                # only neighbour is to the left
                neighbour_set = check_neighbours((0, space[1] - 1), field, neighbour_set)
            else:
                neighbour_set = check_neighbours((space[0], space[1]-1), field, neighbour_set)
                neighbour_set = check_neighbours((space[0], space[1]+1), field, neighbour_set)
                neighbour_set = check_neighbours((space[0]+1, space[1]), field, neighbour_set)
        else:
            # in burrow - can only go up or down
            if space[0] < field.shape[0] - 1:
                neighbour_set = check_neighbours((space[0]-1, space[1]), field, neighbour_set)
                neighbour_set = check_neighbours((space[0]+1, space[1]), field, neighbour_set)
            else:
                # print("this space is in a burrow haha")
                # in bottom of burrow - can only go up
                neighbour_set = check_neighbours((space[0]-1, space[1]), field, neighbour_set)
        return neighbour_set

In [33]:
# Define a class for amphipods to use
class Amphipod:
    def __init__(self, position, species):
        self.y = position[0]
        self.x = position[1]
        self.species = species  # either "A", "B", "C", or "D"
        self.energy_cost = {"A": 1, "B": 10, "C": 100, "D": 1000}[species]
        self.num_to_species = {1: "A", 2: "B", 3: "C", 4: "D"}
        self.species_to_burrow = {"A": 2, "B": 4, "C": 6, "D": 8}
        
        
    def gen_moves(self, field):
        # return a list of all spaces it can move to
        if self.is_endgame(field):
            # if it shouldn't move at all
            return set()
        
        neighbour_set = set()
        # print("this field:")
        # print(field)

        for i in (-1, +1):
            # print(f"checking neighbours for {(self.y+i, self.x)}")
            new_neighbour_set = check_neighbours((self.y+i, self.x), field, neighbour_set)
            # print("new moves:", new_neighbour_set)
            neighbour_set = neighbour_set | new_neighbour_set
            # print("current moves:", neighbour_set)
            
        for i in (-1, +1):
            # print(f"checking neighbours for {(self.y, self.x+j)}")
            new_neighbour_set = check_neighbours((self.y, self.x+i), field, neighbour_set)
            neighbour_set = neighbour_set | new_neighbour_set
            # print("current moves:", neighbour_set)
            
        # print("all empty spaces:", neighbour_set)
        
        # "clean" the possible moves
        # can't stay in same place
        try:
            neighbour_set.remove((self.y, self.x))
        except KeyError:
            pass
        
        # can't stop in front of burrow
        burrow_entrances = ((0, 2), (0, 4), (0, 6), (0, 8))  
        for i in burrow_entrances:
            try:
                neighbour_set.remove(i)
            except KeyError:
                pass
        # print("no burrow entrances:", neighbour_set)
        
        # if in hallway, can't move except to its own burrow
        burrow_column = self.species_to_burrow[self.species]
        if self.y == 0:
            to_remove = []
            for space in neighbour_set:
                if space[0] == 0 or space[1] != burrow_column:
                    to_remove.append(space)
            for space in to_remove:
                neighbour_set.remove(space)
        # print("in hallway:", neighbour_set)
                
        # can't move to any burrow that's not its own ever
        to_remove = []
        for space in neighbour_set:
            if space[0] > 0 and space[1] != burrow_column:
                to_remove.append(space)
        for space in to_remove:
            neighbour_set.remove(space)
        # print("no unowned burrow:", neighbour_set)
            
        # can only move to its own burrow if it's empty or has only its own species
        own_burrow = field[1:, burrow_column]
        # if np.all(own_burrow == "."):
        #     pass
        # # need some logic here - if there is anything but . and species here
        # elif own_burrow[0] == "." and own_burrow[1] == self.species:
        #     pass
        if np.all((own_burrow == ".") | (own_burrow == self.species)):
            pass
        else:
            to_remove = []
            for space in neighbour_set:
                if space[0] > 0 and space[1] == burrow_column:
                    to_remove.append(space)
            for space in to_remove:
                neighbour_set.remove(space)
        # print("no own burrow in some cases:", neighbour_set)
                
        # if it can still move to its own column, then do that
        for i in range(field.shape[0] - 1, 0, -1):
            if (i, burrow_column) in neighbour_set and field[i, burrow_column] == ".":
                neighbour_set = {(i, burrow_column)}
                break
        # print("can move to own column?:", neighbour_set)
            
        # that's all the logic I think?
        return neighbour_set
    
    
    def moveto(self, position):
        "Changes position of amphipod. Returns cost of moving to that space."
        # CURRENTLY WRONG - MANHATTAN DISTANCE TAKES SHORTCUT
        # FIX?: steps = distance to y=0 + distance to (x = position[1]) + distance to y=position[0]
        up_steps = self.y
        across_steps = abs(self.x - position[1])
        down_steps = position[0]

        energy_cost = (up_steps + across_steps + down_steps) * self.energy_cost
        
        self.y = position[0]
        self.x = position[1]
        return energy_cost
    
    
    def is_endgame(self, field):
        "Returns True if this amphipod shouldn't move again, False otherwise."
        burrow_column = self.species_to_burrow[self.species]
        if self.x == burrow_column:  # is at bottom of its burrow
            if self.y == field.shape[0] - 1:
                return True
            # elif field[2, burrow_column] == self.species and self.y == 1:  # or at top of burrow but same species is at bottom
            #     return True
            elif np.all(field[self.y:, self.x] == self.species) and self.y > 0:
                return True
        else:
            return False

In [424]:
test = np.array([1, 2, 1])
print(test == 1)
print(test == 2)
print(np.all((test == 1) | (test == 2)))

for i in range(3, 0, -1):
    print(i)

[ True False  True]
[False  True False]
True
3
2
1


In [425]:
import numpy as np

lines = []
with open("ex1.txt") as f:
    for line in f.readlines():
        line_list = [char for char in line.strip()]
        if lines:
            while len(line_list) < len(lines[0]):
                line_list = ["#"] + line_list + ["#"]
        lines.append(line_list)
    
print(lines)

field = np.array(lines)
print(field)

field = field[1:-1, 1:-1]
print(field)

[['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'], ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#'], ['#', '#', '#', 'B', '#', 'C', '#', 'B', '#', 'D', '#', '#', '#'], ['#', '#', '#', 'A', '#', 'D', '#', 'C', '#', 'A', '#', '#', '#'], ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#']]
[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '#']
 ['#' '#' '#' 'B' '#' 'C' '#' 'B' '#' 'D' '#' '#' '#']
 ['#' '#' '#' 'A' '#' 'D' '#' 'C' '#' 'A' '#' '#' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]
[['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['#' '#' 'B' '#' 'C' '#' 'B' '#' 'D' '#' '#']
 ['#' '#' 'A' '#' 'D' '#' 'C' '#' 'A' '#' '#']]


In [32]:
def get_burrow(field, species):
    "Returns an array of the burrow for species showing its state. Left is top, right is bottom."
    return {"A": field[1:, 2], "B": field[1:, 4], "C": field[1:, 6], "D": field[1:, 8]}[species]

# print(get_burrow(field, "D"))

In [38]:
print(field.tobytes())  # use this to hash a game state

b'.\x00\x00\x00.\x00\x00\x00.\x00\x00\x00.\x00\x00\x00.\x00\x00\x00.\x00\x00\x00.\x00\x00\x00.\x00\x00\x00.\x00\x00\x00.\x00\x00\x00.\x00\x00\x00#\x00\x00\x00#\x00\x00\x00B\x00\x00\x00#\x00\x00\x00C\x00\x00\x00#\x00\x00\x00B\x00\x00\x00#\x00\x00\x00D\x00\x00\x00#\x00\x00\x00#\x00\x00\x00#\x00\x00\x00#\x00\x00\x00A\x00\x00\x00#\x00\x00\x00D\x00\x00\x00#\x00\x00\x00C\x00\x00\x00#\x00\x00\x00A\x00\x00\x00#\x00\x00\x00#\x00\x00\x00'
8045678840571455092


In [26]:
import copy
import time

SPECIES_TO_BURROW = {"A": 2, "B": 4, "C": 6, "D": 8}

def play_move(field, amphipods, amp_to_move, move, best_guess, energy_cost, game_state_cost, moves):
    "Returns best_guess and game_state_cost for future stuff :/"
    # Make copies of things so recursion works fine
    field = field.copy()
    amphipods = copy.deepcopy(amphipods)
    # moves = copy.deepcopy(moves)
    # energy_cost = copy.copy(energy_cost)
    
    # print(f"Move {amphipods[amp_to_move].species} from {(amphipods[amp_to_move].y, amphipods[amp_to_move].x)} to {move}")
    
    # Make move
    if move is not None:
        this_amp = amphipods[amp_to_move]
        field[this_amp.y, this_amp.x] = "."  # old position is now empty
        # print(f"move {this_amp.species} from {(this_amp.y, this_amp.x)} to {move}")
        energy_to_move = amphipods[amp_to_move].moveto(move)
        # print(f"energy to do that = {energy_to_move}")
        energy_cost += energy_to_move
        field[amphipods[amp_to_move].y, amphipods[amp_to_move].x] = amphipods[amp_to_move].species  # new position is now full
        # moves.append((field, energy_cost))
    
    # print(f"after move (energy cost = {energy_cost}):")
    # print(field)
    # time.sleep(0.5)
    
    # Check if need to stop early    
    if energy_cost >= best_guess:
        # Have already spent more energy than allowed, so this can't be a good way of doing it
        return best_guess, game_state_cost
    
    # State is concrete now, so hash it and save cost
    field_bytes = field.tobytes()
    if field_bytes in game_state_cost:
        if game_state_cost[field_bytes] <= energy_cost:
            # print("have reached state I've reached before hmmm")
            return best_guess, game_state_cost  # have already reached this state with a better cost already
    else:
        game_state_cost[field_bytes] = energy_cost
        
    # check if game is over
    if check_won(field):
        print("Game is over!!!")
        print("energy_cost", energy_cost, "best_guess", best_guess)
        
        # if energy_cost < 12521:
        #     print("all moves:")
        #     for i in moves:
        #         print(i[0])  # the field
        #         print("cost so far:", i[1])  # the cost
        #     raise Exception("Stop please")
            
        if energy_cost < best_guess:
            best_guess = energy_cost
        return best_guess, game_state_cost
    
    # print("Game isn't over")
        
    # this game state still could be worthwhile exploring
    # so play every possible move from here
    # then inspect those moves
    for amp_to_move, amp in enumerate(amphipods):
        possible_moves = amp.gen_moves(field)  # all possible positions this amphipod could move to
        # if not possible_moves:
        #     print(f"No more moves possible for {amp.species} at {(amp.y, amp.x)}!")
        # print()
        for move in possible_moves:
            # Inspect that move
            best_guess, game_state_cost = play_move(field, amphipods, amp_to_move, move, best_guess, energy_cost, game_state_cost, moves)
            
    return best_guess, game_state_cost

In [34]:
import collections
import numpy as np
num_to_species = {1: "A", 2: "B", 3: "C", 4: "D"}

lines = []
with open("ex1.txt") as f:
# with open("amphipods.txt") as f:
    hallway_index = 1000
    burrows = collections.defaultdict(list)
    burrow_connection = {}
    for index, line in enumerate(f.readlines()):
        # create field
        line_list = [char for char in line.strip()]
        if lines:
            while len(line_list) < len(lines[0]):
                line_list = ["#"] + line_list + ["#"]
        lines.append(line_list)

        # create other stuff
        line = line.strip()
        if "." in line:
            hallway_index = index
            hallway = ["."] * line.count(".")
        if index > hallway_index:
            n_burrow = 1
            for char_index, char in enumerate(line):
                if char != "#":
                    if n_burrow not in burrow_connection:
                        burrow_connection[num_to_species[n_burrow]] = char_index - 1
                    burrows[num_to_species[n_burrow]].append(char)
                    n_burrow += 1
                
    
field = np.array(lines)
field = field[1:-1, 1:-1]
print(field)

[['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['#' '#' 'B' '#' 'C' '#' 'B' '#' 'D' '#' '#']
 ['#' '#' 'A' '#' 'D' '#' 'C' '#' 'A' '#' '#']]


In [35]:
# field = np.array(
#     [['.', 'A', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
#      ['#', '#', '.', '#', 'B', '#', 'C', '#', 'D', '#', '#'],
#      ['#', '#', 'A', '#', 'B', '#', 'C', '#', 'D', '#', '#']])

amphipods = []
for index, element in np.ndenumerate(field):
    if element in ("ABCD"):
        amphipods.append(Amphipod(index, element))
        
print(field)
# for i in amphipods:
#     print(i.species, (i.y, i.x))
#     print(i.gen_moves(field))
        
best_guess = 19115  # worked out by hand, AoC says it's too high
energy_cost = 0
game_state_cost = {}
moves = [(field, 0)]

# 8301 is too low :(
# Should get 12521
# Hmm, what's going wrong then?
# Calculating energy cost wrong? - checked some, looks like it's doing it right D:
# Making illegal moves? Probably :(

best_guess, game_state_cost = play_move(field, amphipods, 0, None, best_guess, energy_cost, game_state_cost, moves)
print(best_guess)

# wait why has it stopped working...
# D: AAAAA

[['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['#' '#' 'B' '#' 'C' '#' 'B' '#' 'D' '#' '#']
 ['#' '#' 'A' '#' 'D' '#' 'C' '#' 'A' '#' '#']]
Game is over!!!
energy_cost 14721 best_guess 19115
Game is over!!!
energy_cost 12741 best_guess 14721
Game is over!!!
energy_cost 12721 best_guess 12741
Game is over!!!
energy_cost 12601 best_guess 12721
Game is over!!!
energy_cost 12599 best_guess 12601
Game is over!!!
energy_cost 12583 best_guess 12599
Game is over!!!
energy_cost 12541 best_guess 12583
Game is over!!!
energy_cost 12539 best_guess 12541
Game is over!!!
energy_cost 12521 best_guess 12539
12521


In [314]:
field = np.array(
    [['.', 'A', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
     ['#', '#', '.', '#', 'B', '#', 'C', '#', 'D', '#', '#'],
     ['#', '#', 'A', '#', 'B', '#', 'C', '#', 'D', '#', '#']])
print(check_won(field))

field = np.array(
    [['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
     ['#', '#', 'A', '#', 'B', '#', 'C', '#', 'D', '#', '#'],
     ['#', '#', 'A', '#', 'B', '#', 'C', '#', 'D', '#', '#']])
print(check_won(field))

False
True


In [299]:
test_field = np.array(
    [['.', 'B', '.', '.', '.', 'B', '.', '.', '.', '.', '.'],
     ['#', '#', '.', '#', '.', '#', 'C', '#', 'D', '#', '#'],
     ['#', '#', 'A', '#', 'D', '#', 'C', '#', 'A', '#', '#']])

print(test_field)

test_amphipod = Amphipod((2, 4), "D")
print(f"test_amphipod is in position {(test_amphipod.y, test_amphipod.x)}")
poss_moves = test_amphipod.gen_moves(test_field)
print("possible moves:")
for i in poss_moves:
    print(i)
print(test_amphipod.is_endgame(test_field))
print()
# why isn't it generating any moves?

neighbours = set()
test_pos = (1, 4)
print(f"check neighbours for {test_pos}:")
checked_neighbours = check_neighbours(test_pos, test_field, neighbours)
print(checked_neighbours)

[['.' 'B' '.' '.' '.' 'B' '.' '.' '.' '.' '.']
 ['#' '#' '.' '#' '.' '#' 'C' '#' 'D' '#' '#']
 ['#' '#' 'A' '#' 'D' '#' 'C' '#' 'A' '#' '#']]
test_amphipod is in position (2, 4)
possible moves:
(0, 3)
False

check neighbours for (1, 4):
{(1, 2), (0, 4), (0, 3), (1, 4), (0, 2)}


In [300]:
test_field = np.array(
    [['.', 'B', '.', '.', '.', 'B', '.', '.', '.', '.', '.'],
     ['#', '#', '.', '#', '.', '#', 'C', '#', 'D', '#', '#'],
     ['#', '#', 'A', '#', '.', '#', 'C', '#', 'A', '#', '#']])

print(test_field)

test_amphipod = Amphipod((0, 1), "B")
print(f"test_amphipod is in position {(test_amphipod.y, test_amphipod.x)}, species {test_amphipod.species}")
poss_moves = test_amphipod.gen_moves(test_field)
print("possible moves:")
for i in poss_moves:
    print(i)
print("endgame?", test_amphipod.is_endgame(test_field))
print()
# why isn't it generating any moves?

# neighbours = set()
# test_pos = (1, 4)
# print(f"check neighbours for {test_pos}:")
# checked_neighbours = check_neighbours(test_pos, test_field, neighbours)
# print(checked_neighbours)

[['.' 'B' '.' '.' '.' 'B' '.' '.' '.' '.' '.']
 ['#' '#' '.' '#' '.' '#' 'C' '#' 'D' '#' '#']
 ['#' '#' 'A' '#' '.' '#' 'C' '#' 'A' '#' '#']]
test_amphipod is in position (0, 1), species B
possible moves:
(2, 4)
endgame? False



In [336]:
test_field = np.array(
    [['.', 'A', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
     ['#', '#', '.', '#', 'B', '#', 'C', '#', 'D', '#', '#'],
     ['#', '#', 'A', '#', 'B', '#', 'C', '#', 'D', '#', '#']])

test_amphipod = Amphipod((0, 1), "A")
print(f"test_amphipod is in position {(test_amphipod.y, test_amphipod.x)}, species {test_amphipod.species}")
poss_moves = test_amphipod.gen_moves(test_field)
print("possible moves:")
for i in poss_moves:
    print(i)
print("endgame?", test_amphipod.is_endgame(test_field))
print()

test_amphipod is in position (0, 1), species A
all empty spaces: {(0, 7), (1, 2), (0, 4), (0, 10), (0, 0), (0, 3), (0, 9), (0, 6), (0, 2), (0, 5), (0, 8)}
no burrow entrances: {(0, 7), (1, 2), (0, 10), (0, 0), (0, 3), (0, 9), (0, 5)}
in hallway: {(1, 2)}
possible moves:
(1, 2)
endgame? False



## Part 2

In [38]:
import collections
import numpy as np
num_to_species = {1: "A", 2: "B", 3: "C", 4: "D"}

lines = []
# with open("ex2.txt") as f:
with open("amphipods2.txt") as f:
    hallway_index = 1000
    burrows = collections.defaultdict(list)
    burrow_connection = {}
    for index, line in enumerate(f.readlines()):
        # create field
        line_list = [char for char in line.strip()]
        if lines:
            while len(line_list) < len(lines[0]):
                line_list = ["#"] + line_list + ["#"]
        lines.append(line_list)

        # create other stuff
        line = line.strip()
        if "." in line:
            hallway_index = index
            hallway = ["."] * line.count(".")
        if index > hallway_index:
            n_burrow = 1
            for char_index, char in enumerate(line):
                if char != "#":
                    if n_burrow not in burrow_connection:
                        burrow_connection[num_to_species[n_burrow]] = char_index - 1
                    burrows[num_to_species[n_burrow]].append(char)
                    n_burrow += 1
                
    
field = np.array(lines)
field = field[1:-1, 1:-1]
print(field)

[['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['#' '#' 'D' '#' 'C' '#' 'B' '#' 'C' '#' '#']
 ['#' '#' 'D' '#' 'C' '#' 'B' '#' 'A' '#' '#']
 ['#' '#' 'D' '#' 'B' '#' 'A' '#' 'C' '#' '#']
 ['#' '#' 'D' '#' 'A' '#' 'A' '#' 'B' '#' '#']]


In [39]:
# field = np.array(
#     [['.', 'A', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
#      ['#', '#', '.', '#', 'B', '#', 'C', '#', 'D', '#', '#'],
#      ['#', '#', 'A', '#', 'B', '#', 'C', '#', 'D', '#', '#']])

amphipods = []
for index, element in np.ndenumerate(field):
    if element in ("ABCD"):
        amphipods.append(Amphipod(index, element))
        
print(field)
# for i in amphipods:
#     print(i.species, (i.y, i.x))
#     print(i.gen_moves(field))
        
best_guess = 50000  # worked out by hand, AoC says it's too high
energy_cost = 0
game_state_cost = {}
moves = [(field, 0)]

# Should get 44169 for example 2

best_guess, game_state_cost = play_move(field, amphipods, 0, None, best_guess, energy_cost, game_state_cost, moves)
print(best_guess)

# wait why has it stopped working...
# D: AAAAA

[['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['#' '#' 'D' '#' 'C' '#' 'B' '#' 'C' '#' '#']
 ['#' '#' 'D' '#' 'C' '#' 'B' '#' 'A' '#' '#']
 ['#' '#' 'D' '#' 'B' '#' 'A' '#' 'C' '#' '#']
 ['#' '#' 'D' '#' 'A' '#' 'A' '#' 'B' '#' '#']]
Game is over!!!
energy_cost 49303 best_guess 50000
Game is over!!!
energy_cost 49301 best_guess 49303
Game is over!!!
energy_cost 48583 best_guess 49301
Game is over!!!
energy_cost 48581 best_guess 48583
Game is over!!!
energy_cost 48543 best_guess 48581
Game is over!!!
energy_cost 48541 best_guess 48543
48541


What if I implement a "minimum score from current state to win" calculation? Basicaly, give every amphipod a desired destination and after every move, work out what the energy cost would be to move every amphipod straight to its desired destination. If I know that the current score plus the minimum score to win is greater than the best guess, then I know that from the current state I could never beat the best guess, so return early. Could this save a lot of time???

How would I do that?
1. Ignore any amphipods where amp.is_endgame() == True. This makes sure it doesn't waste time trying to calculate a score for those that are already done.
2. For every type of amphipod, work out what locations are left for it to go that would fit endgame. This could be done by inspecting the burrows and then just finding the coordinates of the points which don't have an endgame amphipod in.
3. Maybe instead, get the coordinates of each amphipod that *is* endgame for each species, and use the highest up one for each species as a lower limit.
3. Work out the theoretical energy score to move each amphipod to a location that is endgame.
4. Energy score + theoretical energy score: work out if this is greater than best_case. If so, return.